# 아산병원 증상 정보 크롤링 + 자연어분석

In [8]:
import pandas as pd
import numpy as np

In [9]:
# CSV 파일 경로
file_path = './../Model/Data/diseases_240510.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)
df = df.dropna(subset=['symptoms'])

In [13]:
# 모든 증상코드를 저장할 빈 집합을 만듭니다. 집합은 중복된 값을 저장하지 않습니다.
all_codes_set = set()

# 각 행(질병)에 대해 반복
for row in df['symptom_ids']:
    codes = row.split(',')
    all_codes_set.update(codes)

all_codes_list = list(all_codes_set)

In [24]:
import requests
from bs4 import BeautifulSoup
import csv

# CSV 파일에 저장할 헤더
header = ['symptoms', 'symptom_ids', 'symptom_describe']

# CSV 파일 열기
with open('symptom_description.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)

    # 질병 종류별 순환
    symptom_ids_list = all_codes_list
    
    for id in symptom_ids_list:  # 질병 종류는 C000001부터 C000020까지 순환
        try: 
            url = f'https://www.amc.seoul.kr/asan/healthinfo/symptom/symptomDetail.do?symptomId={id}'
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
    
            
            symptoms = soup.select_one('h1 > span').text
            symptom_ids = id
            symptom_describe = soup.select_one('#popupWrap > div.popContent > div').text
    
           
            # CSV 파일에 추가
            writer.writerow([symptoms, symptom_ids, symptom_describe])
            print(id, " 끝")
        except:
            print(id, e)
    
print("CSV 파일이 성공적으로 생성되었습니다.")


SS000400  끝
SS000270  끝
SS000607  끝
SS000815  끝
SS000132  끝
SS000065  끝
SS000436  끝
SS000193  끝
SS000147  끝
SS000913  끝
SS000566  끝
SS000289  끝
SS000398  끝
SS000542  끝
SS000174  끝
SS000021  끝
SS000218  끝
SS000763  끝
SS000386  끝
SS000522  끝
SS000210  끝
SS000004  끝
SS000799  끝
SS000801  끝
SS000485  끝
SS000070  끝
SS000298  끝
SS000405  끝
SS000376  끝
SS000363  끝
SS000750  끝
SS000126  끝
SS000593  끝
SS000046  끝
SS000091  끝
SS000589  끝
SS000683  끝
SS000446  끝
SS000428  끝
SS000112  끝
SS000414  끝
SS000256  끝
SS000179  끝
SS000107  끝
SS000700  끝
SS000339  끝
SS000059  끝
SS000312  끝
SS000477  끝
SS000003  끝
SS000397  끝
SS000792  끝
SS000862  끝
SS000748  끝
SS000812  끝
SS000108  끝
SS000845  끝
SS000281  끝
SS000638  끝
SS000033  끝
SS000554  끝
SS000491  끝
SS000895  끝
SS000704  끝
SS000333  끝
SS000426  끝
SS000610  끝
SS000561  끝
SS000773  끝
SS000821  끝
SS000694  끝
SS000284  끝
SS000075  끝
SS000300  끝
SS000744  끝
SS000514  끝
SS000236  끝
SS000543  끝
SS000034  끝
SS000535  끝
SS000517  끝
SS000395  끝
SS000329  끝
SS00

In [33]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# CSV 파일 로드
df = pd.read_csv('symptom_description.csv')  # 'symptoms.csv'는 CSV 파일의 경로

# NaN 값을 빈 문자열로 대체
df['symptom_describe'] = df['symptom_describe'].fillna('')

# TfidfVectorizer 초기화 및 증상 설명에 대한 TF-IDF 행렬 생성
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['symptom_describe'])

def find_similar_symptoms(user_input, top_n=3):
    # 사용자 입력을 TF-IDF 벡터로 변환
    user_input_tfidf = vectorizer.transform([user_input])
    
    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()
    
    # 가장 유사도가 높은 상위 N개 증상의 인덱스 찾기
    top_n_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 가장 유사한 증상과 그 유사도 반환
    similar_symptoms = [(df.iloc[idx]['symptoms'], cosine_similarities[idx]) for idx in top_n_indices]
    return similar_symptoms

# 사용자 입력
user_input = "잠에서 깨면 전에 일이 기억안남"

# 가장 유사한 증상 찾기
similar_symptoms = find_similar_symptoms(user_input)
for symptom, similarity in similar_symptoms:
    print(f"증상: {symptom}, 유사도: {similarity:.2f}")


증상: 몽유병, 유사도: 0.53
증상: 무균성 뇌막염, 유사도: 0.00
증상: 악취가 나는 설사, 유사도: 0.00


In [1]:
# TfidfVectorizer 초기화 및 증상 설명에 대한 TF-IDF 행렬 생성
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['symptom_describe'])

def find_similar_symptoms(user_input, top_n=3):
    # 사용자 입력을 TF-IDF 벡터로 변환
    user_input_tfidf = vectorizer.transform([user_input])
    
    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()
    
    # 가장 유사도가 높은 상위 N개 증상의 인덱스 찾기
    top_n_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 가장 유사한 증상, 설명, 그리고 유사도 반환
    similar_symptoms = []
    for idx in top_n_indices:
        if cosine_similarities[idx] > 0:  # 유사도가 0보다 클 때만 결과에 포함
            symptom = df.iloc[idx]['symptoms']
            description = df.iloc[idx]['symptom_describe']
            similarity = cosine_similarities[idx]
            similar_symptoms.append((symptom, description, similarity))
    return similar_symptoms

# 사용자 입력
user_input = "피부가 두꺼워짐"

# 가장 유사한 증상 찾기
similar_symptoms = find_similar_symptoms(user_input)
for symptom, description, similarity in similar_symptoms:
    print(f"증상: {symptom}, 설명: {description}, 유사도: {similarity:.2f}")


NameError: name 'TfidfVectorizer' is not defined

In [43]:
# TfidfVectorizer 초기화 및 증상 설명에 대한 TF-IDF 행렬 생성
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['symptom_describe'])

def find_similar_symptoms(user_input):
    # 사용자 입력을 TF-IDF 벡터로 변환
    user_input_tfidf = vectorizer.transform([user_input])
    
    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()
    
    # 유사도가 0보다 큰 증상의 인덱스와 유사도를 함께 반환
    similar_symptoms_indices = cosine_similarities.argsort()[::-1]
    similar_symptoms = [(idx, cosine_similarities[idx]) for idx in similar_symptoms_indices if cosine_similarities[idx] > 0]
    
    # 유사도가 0보다 큰 증상과 그 유사도 반환
    return [(df.iloc[idx]['symptoms'], df.iloc[idx]['symptom_describe'], similarity) for idx, similarity in similar_symptoms]

# 사용자 입력
user_input = "자기 자신이나 사물에 대하여 인식하는 능력이 감소함"

# 가장 유사한 증상 찾기
similar_symptoms = find_similar_symptoms(user_input)
for symptom, description, similarity in similar_symptoms:
    print(f"증상: {symptom}, 설명: {description}, 유사도: {similarity:.2f}")
